## Preprocessing

In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

## What variable(s) are the target(s) for your model?

IS_SUCCESSFUL: This is the target variable, it's a binary variable indicating if a charity project was successful (1) or not (0).

## What variable(s) are the feature(s) for your model?
EIN: Employer Identification Number, a unique identifier.
NAME: The name of the charity organization.
APPLICATION_TYPE: The type of application the charity submitted.
AFFILIATION: The affiliation of the charity (Independent, Company Sponsored, others).
CLASSIFICATION: The classification of the charity.
USE_CASE: The use case category of the charity.
ORGANIZATION: The type of organization (Association, Trust, others).
STATUS: The status of the organization (1 or 0).
INCOME_AMT: The amount of income the organization receives.
SPECIAL_CONSIDERATIONS: Whether there are special considerations for the charity (N or Y).
ASK_AMT: The amount of money the charity is asking for.

- There is a duplicated column name "IS_SUCCESSFUL"

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [3]:
# Determine the number of unique values in each column
unique_value_counts = application_df.nunique()
unique_value_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500

# Create a list of application types to be replaced
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# Calculate the value counts for CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Filter and display value counts greater than 1
classification_counts_gt_1 = classification_counts[classification_counts > 1]
classification_counts_gt_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Choose a cutoff value for classifications
cutoff_value = 500

# Create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in the dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
# Creates a list of all columns that are a string
application_df = pd.get_dummies(application_df, columns=application_df.dtypes[application_df.dtypes == "object"].index.tolist())

# Display the modified DataFrame
application_df.head()

STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                       0   
1       1   108590              1                       0   
2       1     5000              0                       0   
3       1     6692              1                       0   
4       1   142590              1                       0   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                     1                     0                    0   
1                     0                     0                    1   
2                     0                     0                    0   
3                     0                     0                    1   
4                     0                     0                    1   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                    0                    0                    0  ...   
1                    0                    0                    0  ...   
2                    0                    1                    0  ...   
3                    0                    0                    0  ...   
4                    0                    0                    0  ...   

   INCOME_AMT_1-9999  INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  \
0                  0                       0                         0   
1                  1                       0                         0   
2                  0                       0                         0   
3                  0                       1                         0   
4                  0                       0                         1   

   INCOME_AMT_10M-50M  INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  \
0                   0                 0                       0   
1                   0                 0                       0   
2                   0                 0                       0   
3                   0                 0                       0   
4                   0                 0                       0   

   INCOME_AMT_50M+  INCOME_AMT_5M-10M  SPECIAL_CONSIDERATIONS_N  \
0                0                  0                         1   
1                0                  0                         1   
2                0                  0                         1   
3                0                  0                         1   
4                0                  0                         1   

   SPECIAL_CONSIDERATIONS_Y  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0  

[5 rows x 45 columns]

In [10]:
application_df.dtypes

STATUS                          int64
ASK_AMT                         int64
IS_SUCCESSFUL                   int64
APPLICATION_TYPE_Other          uint8
APPLICATION_TYPE_T10            uint8
APPLICATION_TYPE_T19            uint8
APPLICATION_TYPE_T3             uint8
APPLICATION_TYPE_T4             uint8
APPLICATION_TYPE_T5             uint8
APPLICATION_TYPE_T6             uint8
APPLICATION_TYPE_T7             uint8
APPLICATION_TYPE_T8             uint8
AFFILIATION_CompanySponsored    uint8
AFFILIATION_Family/Parent       uint8
AFFILIATION_Independent         uint8
AFFILIATION_National            uint8
AFFILIATION_Other               uint8
AFFILIATION_Regional            uint8
CLASSIFICATION_C1000            uint8
CLASSIFICATION_C1200            uint8
CLASSIFICATION_C2000            uint8
CLASSIFICATION_C2100            uint8
CLASSIFICATION_C3000            uint8
CLASSIFICATION_C7000            uint8
CLASSIFICATION_Other            uint8
USE_CASE_CommunityServ          uint8
USE_CASE_Hea

In [11]:
# Split our preprocessed data into our features and target arrays
# Separate features (X) and target (y)
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (27439, 44)
X_test shape: (6860, 44)
y_train shape: (27439,)
y_test shape: (6860,)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train_scaled[0])

# First hidden layer
nn.add(Dense(units=80, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(Dense(units=30, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3600      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6061 (23.68 KB)
Trainable params: 6061 (23.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the model checkpoint callback
checkpoint_path = "model_checkpoint.h5"
cp_callback = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, save_weights_only=True, verbose=1)

# Train the model
model_history = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback], validation_split=0.1)


Epoch 1/100
749/772 [============================>.] - ETA: 0s - loss: 0.5697 - accuracy: 0.7213
Epoch 1: val_loss improved from inf to 0.54804, saving model to model_checkpoint.h5
772/772 [==============================] - 3s 2ms/step - loss: 0.5703 - accuracy: 0.7210 - val_loss: 0.5480 - val_accuracy: 0.7372
Epoch 2/100
752/772 [============================>.] - ETA: 0s - loss: 0.5545 - accuracy: 0.7275
Epoch 2: val_loss improved from 0.54804 to 0.54313, saving model to model_checkpoint.h5
772/772 [==============================] - 1s 2ms/step - loss: 0.5541 - accuracy: 0.7280 - val_loss: 0.5431 - val_accuracy: 0.7405
Epoch 3/100
764/772 [============================>.] - ETA: 0s - loss: 0.5522 - accuracy: 0.7289
Epoch 3: val_loss improved from 0.54313 to 0.54048, saving model to model_checkpoint.h5
772/772 [==============================] - 1s 2ms/step - loss: 0.5518 - accuracy: 0.7291 - val_loss: 0.5405 - val_accuracy: 0.7394
Epoch 4/100
764/772 [============================>.] - E

772/772 [==============================] - 1s 1ms/step - loss: 0.5360 - accuracy: 0.7392 - val_loss: 0.5409 - val_accuracy: 0.7376
Epoch 60/100
740/772 [===========================>..] - ETA: 0s - loss: 0.5360 - accuracy: 0.7386
Epoch 60: val_loss did not improve from 0.53694
772/772 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7387 - val_loss: 0.5372 - val_accuracy: 0.7380
Epoch 61/100
737/772 [===========================>..] - ETA: 0s - loss: 0.5365 - accuracy: 0.7388
Epoch 61: val_loss did not improve from 0.53694
772/772 [==============================] - 1s 2ms/step - loss: 0.5362 - accuracy: 0.7393 - val_loss: 0.5389 - val_accuracy: 0.7398
Epoch 62/100
737/772 [===========================>..] - ETA: 0s - loss: 0.5352 - accuracy: 0.7396
Epoch 62: val_loss did not improve from 0.53694
772/772 [==============================] - 1s 2ms/step - loss: 0.5358 - accuracy: 0.7391 - val_loss: 0.5449 - val_accuracy: 0.7383
Epoch 63/100
749/772 [================

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5674 - accuracy: 0.7264 - 235ms/epoch - 1ms/step
Loss: 0.5673836469650269, Accuracy: 0.7263848185539246


In [25]:
# Export our model to HDF5 file
model_filename = "AlphabetSoupCharity_Optimization.h5"
nn.save(model_filename)